In [ ]:
import gmsh

import sys
%matplotlib inline
import matplotlib.pyplot as plt
from sympy import solve, symbols
import cmath
from math import pi, sin, cos, asin, log10, log2
import numpy as np
import pandas as pd
import altair as alt

from firedrake import *
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

import sys,os,os.path
sys.path.append(os.path.expanduser('/home/pierre/src/spinorama/src'))
from spinorama.graph import graph_contour, contour_params_default, graph_isoband, isoband_params_default
from spinorama.load_misc import sort_angles, graph_melt

In [ ]:
def plot_contour(freq, sol):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,6))
    levels= np.linspace(-1, 1, 21)
    # levels = 21
    ax1.set_aspect('equal')
    # ax1.legend()
    colors= tripcolor(sol, axes=ax1, cmap="inferno")
    fig.colorbar(colors, ax=ax1)
    ax2.set_aspect('equal')
    contours = tricontour(sol, axes=ax2, levels=levels, cmap="inferno")
    fig.colorbar(contours)
    plt.title('{}Hz'.format(freq))
    plt.show()

In [ ]:
# speed of sound in air (m/s)
c_air = 343.344
# 
rho_air = 1.20458
# incoming wave amplitude
p_A_in = 1
# outgoing wave amplitude
p_B_out = 1
# radius of outer circle
p_R_omega = 1
# 2 case, plannar and cylindrical
# plannar
p_kappa = 2
p_r = 1
# cyclindrical
# kappa = 1
# r = distance to symmetric axis

In [ ]:
# mesh = Mesh('horn-line.msh')
mesh = Mesh('horn-curve.msh')
g_inlet = 2
g_outlet = 3
# length of the boundaries
l_inlet = 1/15

In [ ]:
#mesh = UnitSquareMesh(100, 100)
#g_inlet  = 1
#g_outlet = 3

def helmotz_complex(mesh, freq):
    V = FunctionSpace(mesh, "CG", 1)
    u = TrialFunction(V)
    v = TestFunction(V)
    kappa = 2*pi*freq/c_air
    a_prod = inner(grad(u), grad(v)) - kappa**2*inner(u, v)
    # right hand side
    a_in  =  rho_air*1j*kappa     *inner(u, v)
    a_out = (rho_air*1j*kappa+1/2)*inner(u, v)
    a = a_prod * dx + a_in * ds(g_inlet) + a_out *ds(g_outlet)
    L = -2*rho_air*1j*kappa*conj(v)*ds(g_inlet)
    # solve
    assemble(a)
    assemble(L)
    sol = Function(V)
    solve(a == L, sol, solver_parameters={'ksp_type': 'fgmres'})
    return sol

In [ ]:
sol900 = helmotz_complex(mesh, 900)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20,10))
levels= np.linspace(-1.1, 1.1, 51)
ax1.set_aspect('equal')
colors= tripcolor(sol900, axes=ax1, cmap="inferno")
fig.colorbar(colors, ax=ax1)
ax2.set_aspect('equal')
contours = tricontour(sol900, axes=ax2, levels=levels, cmap="inferno")
fig.colorbar(contours)
plt.title('900 Hz')
plt.show()

In [ ]:
out = File("horn1.pvd")
out.write(sol900)

In [ ]:
quick_results = {}
for hz in (50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 1000):
    sol_hz = helmotz_complex(mesh, hz)
    quick_results[hz] = sol_hz

In [ ]:
for f_hz, res in quick_results.items():
    plot_contour(f_hz, res)

In [ ]:
j = {}
for freq, sol in quick_results.items():
    intermediate = assemble(sol*ds(g_inlet))
    length = assemble(sol/sol*ds(g_inlet))
    print(intermediate, length)
    j[freq] = abs( intermediate/length - 1)
print(j)

In [ ]:
freq_sz = 101
freq = np.logspace(log10(50), log10(1050), freq_sz) #np.linspace(50, 1000, freq_sz)
results = []
for i, f_hz in enumerate(freq):
    s_hz = helmotz_complex(mesh, f_hz)
    results.append(s_hz)
    if i>0 and (i*100//freq_sz) % 10 == 0:
        print('{}%'.format(i*100//freq_sz), end='')
    if i % 4 == 0:
      print('.', end='')

In [ ]:
length_in = abs(assemble(results[0]/results[0]*ds(g_inlet)))
print(length_in)
reflections = [abs(abs(assemble(abs(s_hz)*ds(g_inlet))/length_in)-1) for s_hz in results]

alt.Chart(pd.DataFrame({'Freq': freq, 'Reflection': reflections})).mark_line(
).encode(
    x=alt.X('Freq', title='Freq (Hz)', scale=alt.Scale(type="linear", base=10, nice=False, domain=[20, 1050])), 
    y=alt.Y('Reflection', scale=alt.Scale(type="log", base=10, nice=False, domain=[0.001, 2])),
)

In [ ]:
def angle(theta):
    if theta == 0:
        return 'On Axis'
    return '{}°'.format(int(theta))

r = 0.99
dfs = []
for theta in np.linspace(0, 180, 19): # use s/19/37/ for 5 degrees increment
    dbs = []
    theta_rad = theta*pi/180
    p_x = r*cos(theta_rad)
    p_y = r*sin(theta_rad)
    for fr, sol in zip(freq, results):
        p_p = sol.at(p_x, p_y, dont_raise=True)
        if p_p is not None and abs(p_p)>=0:
            if abs(p_p) == 0:
                p_db = -120
            else:
                p_db = 105+20*log10(abs(p_p))
            # print('{:3.0f} {:+0.2f} {:+0.2f} {}'.format(theta, p_x, p_y, p_db))
            dbs.append(p_db)
        else:
            print('{} {} {} {} ERROR'.format(theta, p_x, p_y, p_p))
    if theta == 0:
        dfs.append(pd.DataFrame({"Freq": freq, angle(theta): dbs}))
    else:
        angle_p = angle(theta)
        angle_m = '-{}'.format(angle_p)
        dfs.append(pd.DataFrame({angle_p: dbs}))
        if theta != 180:
            dfs.append(pd.DataFrame({angle_m: dbs}))

df = sort_angles(pd.concat(dfs, axis=1))
dfs = graph_melt(df)

In [ ]:
alt.Chart(dfs).mark_line(
).transform_filter(
    alt.FieldOneOfPredicate(
        field="Measurements",
        oneOf=[
            "On Axis",
            "10°",
            "20°",
            "30°",
            "40°",
            "50°",
            "60°",
            "90°",
            "120°",
            "150°",
        ],),
).encode(
    x=alt.X('Freq:Q', scale=alt.Scale(type="log", base=10, nice=False, domain=[50, 1000])),
    y=alt.Y('dB:Q', scale=alt.Scale(type="linear", domain=[60, 110])),
    color=alt.Color('Measurements')
).properties(
    width=600
)

In [ ]:
isoband_params = isoband_params_default
isoband_params['xmin'] = 50
isoband_params['xmax'] = 1000
graph_isoband(df, isoband_params)

In [ ]:
from spinorama.compute_cea2034 import compute_cea2034

from spinorama.graph import graph_spinorama, graph_params_default
from spinorama.compute_normalize import normalize_mean, normalize_cea2034, normalize_graph

In [ ]:
spin = compute_cea2034(df, df)
mean = normalize_mean(spin)
n_spin = normalize_cea2034(graph_melt(spin), mean)
graph_spinorama(n_spin, graph_params_default)

In [ ]:
m = UnitSquareMesh(20, 20)
mesh = ExtrudedMesh(m, layers=10, layer_height=0.02)

In [ ]:
triplot(mesh)